##Data Cleansing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

url = '/content/drive/MyDrive/WashU/courses/Seminar in Fintech/files/Code/Data_ver2.csv'
df = pd.read_csv(url)
df

In [ ]:
len(df.iloc[:,1:])

35771

In [ ]:
df.iloc[:,4:-3].columns

Index(['currentRatio', 'quickRatio', 'cashRatio', 'daysOfSalesOutstanding',
       'daysOfInventoryOutstanding', 'operatingCycle',
       'daysOfPayablesOutstanding', 'cashConversionCycle', 'grossProfitMargin',
       'operatingProfitMargin', 'pretaxProfitMargin', 'netProfitMargin',
       'effectiveTaxRate', 'returnOnAssets', 'returnOnEquity',
       'returnOnCapitalEmployed', 'netIncomePerEBT', 'ebtPerEbit',
       'ebitPerRevenue', 'debtRatio', 'debtEquityRatio',
       'longTermDebtToCapitalization', 'totalDebtToCapitalization',
       'interestCoverage', 'cashFlowToDebtRatio', 'companyEquityMultiplier',
       'receivablesTurnover', 'payablesTurnover', 'inventoryTurnover',
       'fixedAssetTurnover', 'assetTurnover', 'operatingCashFlowPerShare',
       'freeCashFlowPerShare', 'cashPerShare', 'payoutRatio',
       'operatingCashFlowSalesRatio', 'freeCashFlowOperatingCashFlowRatio',
       'cashFlowCoverageRatios', 'shortTermCoverageRatios',
       'capitalExpenditureCoverageRatio'

In [ ]:
# Transfrom Element in Industry
for i in range(len(df['industry'])):
  if ' ' in df['industry'][i]:
    df['industry'][i] = df['industry'][i].replace(' ','_')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### Find NA in ratios & industry

In [ ]:
# Create a dictionary of mltiples & total NA number
columns = df.columns
multiple_dict = {}
for i in columns:
  try:
    multiple_dict[i] = df[i].isnull().sum()
    
  except:
    continue
multiple_data = pd.DataFrame.from_dict(multiple_dict,orient = 'index' )

In [ ]:
# Create a dictionary of industry and total NA number
industry_dict = {}
for i in df['industry'].unique():
  try:
    row, column = df.loc[df['industry']== i].shape
    industry_dict[i] = row
  except:
    continue


In [ ]:
# Each rows in industry
df['industry'].value_counts()

Industrials               5142
Financial_Services        5059
Technology                4947
Healthcare                4752
Consumer_Cyclical         4445
Consumer_Defensive        2516
Real_Estate               2256
Utilities                 2172
Communication_Services    1593
Basic_Materials           1528
Energy                    1361
Name: industry, dtype: int64

In [ ]:
#Create Table with x = industry, y = multiples and the value = total NA number

Data_result = pd.DataFrame(columns = columns)
for i in df['industry'].unique():
  try:
    temp_data = df.loc[df['industry']== i]
    row, column = temp_data.shape
    multiple_dict_2 = {}
    multiple_dict_2[i] = temp_data.isnull().sum()
    multiple_data_2 = pd.DataFrame.from_dict(multiple_dict_2,orient = 'index')
    #print(multiple_data)

    try:
      Data_result = Data_result.append(multiple_data_2)

    except:
      print("Error rows detacted")
      continue

  except:
    continue
Original_Data = Data_result
Original_Data = Original_Data.iloc[:,4:]
Data_result['Industry_NA']= Data_result.sum(axis=1)
Data_result = Data_result.T
Data_result['Multiples_NA']= Data_result.sum(axis=1)
Data_result = Data_result.T
Data_result = Data_result.iloc[:,4:]

In [ ]:
# Color the NA number when > 0
import pandas as pd
import numpy as np
import matplotlib as mpl

Original_Data.style.applymap(lambda x: "background-color: lightgrey" if x>0 else "background-color: white")


,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,netIncomePerEBT,ebtPerEbit,ebitPerRevenue,debtRatio,debtEquityRatio,longTermDebtToCapitalization,totalDebtToCapitalization,interestCoverage,cashFlowToDebtRatio,companyEquityMultiplier,receivablesTurnover,payablesTurnover,inventoryTurnover,fixedAssetTurnover,assetTurnover,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,payoutRatio,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatio,dividendPaidAndCapexCoverageRatio,dividendPayoutRatio,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,price,trend,industry
Technology,2,2,2,0,3,3,3,3,0,0,0,0,2,2,0,2,2,0,0,2,0,1216,2276,1945,1216,0,9,122,1629,3,2,2,2,2,0,0,0,2276,2076,26,2849,2838,2,2,2,2,2,2,2,2,2,2838,0,2,0,0,0
Communication_Services,0,0,0,0,166,166,166,166,0,0,0,0,12,0,0,0,12,0,0,0,0,265,527,488,265,0,72,85,765,29,0,2,2,2,0,0,0,527,456,15,961,948,2,2,2,2,0,2,2,2,2,948,0,2,0,0,0
Consumer_Cyclical,0,0,0,2,19,21,19,21,2,2,2,2,76,0,0,0,76,4,2,0,0,253,1196,1626,253,0,273,63,409,96,0,0,0,0,5,2,1,1196,1097,35,1715,1687,0,0,2,5,0,1,1,5,2,1683,0,0,0,0,0
Financial_Services,407,407,407,3,3935,3936,3935,3936,3,3,3,3,7,0,0,407,7,3283,3,0,0,306,1726,733,306,0,796,2490,5016,865,0,19,19,19,3,3,2,1726,1695,1026,1402,443,19,19,22,22,2,21,21,22,22,440,4,19,0,0,0
Healthcare,86,86,86,5,302,302,302,302,5,5,5,5,140,0,0,86,140,226,5,0,0,411,984,1735,412,0,11,312,548,3,0,2,2,2,4,5,1,984,881,32,2414,2395,2,2,7,6,2,3,3,6,7,2394,1,2,0,0,0
Consumer_Defensive,0,0,0,5,1,5,1,5,5,5,5,5,1,0,0,0,1,1,5,0,0,67,216,928,67,0,127,68,4,9,0,0,0,0,1,5,9,216,190,13,344,336,0,0,5,1,10,9,9,1,5,335,1,0,0,0,0
Energy,3,3,3,4,42,42,42,42,4,4,4,4,7,3,1,3,7,3,4,3,1,18,234,467,17,1,6,4,78,4,3,0,0,0,3,4,1,233,218,5,90,89,1,1,4,3,0,1,1,3,4,88,3,1,0,0,0
Industrials,0,0,0,41,8,48,8,48,41,41,41,41,3,0,1,0,3,3,41,0,1,315,648,1768,314,1,65,64,948,91,0,5,5,5,4,41,39,647,562,57,1001,951,6,6,46,9,41,44,44,9,46,948,2,6,0,0,0
Basic_Materials,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,100,518,4,0,3,71,0,0,0,2,2,2,0,0,0,100,96,8,55,49,2,2,2,2,3,2,2,2,2,49,0,2,0,0,0
Utilities,0,0,0,0,95,95,95,95,0,0,0,0,7,0,0,0,7,1,0,0,0,0,6,728,0,0,4,19,9,5,0,1,1,1,1,0,0,6,6,60,131,72,1,1,1,2,1,1,1,2,1,71,0,1,0,0,0


In [ ]:
nan_selection = Original_Data.iloc[:,:-3]

In [ ]:
test = Original_Data.iloc[:,:-3]
test2 = test.sum(axis = 1)
test2

Technology                21375.0
Communication_Services     7063.0
Consumer_Cyclical         11854.0
Financial_Services        39913.0
Healthcare                15661.0
Consumer_Defensive         3021.0
Energy                     1814.0
Industrials                9107.0
Basic_Materials            1084.0
Utilities                  1528.0
Real_Estate               12458.0
dtype: float64

In [ ]:
rows, col = df.shape
Total_number = rows * col
Total_number

2182031

In [ ]:
rows, col = df.shape
Total_number = rows * col
nan_rate = Data_result['Industry_NA'][-1]/Total_number
nan_rate

0.05723016767406146

In [ ]:
from google.colab import files
Data_result.to_csv('Data_NA.csv')
files.download("Data_NA.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Select Top 30 ratios with fewest NA based on Industry + Exclude Outlier

In [ ]:
# Filter out top 30 ratios name and append neccessary features

industry_nan = {}
for i in df['industry'].unique():
  Temp_df = nan_selection.loc[i]
  rslt_df = Temp_df.sort_values(ascending = True)   #sort
  rslt_df = rslt_df[:30]    # top30
  Ratios_list = [i for i in rslt_df.index]
  Additional_features = ['trend','industry','date']   #append neccessary features
  All_features = Ratios_list + Additional_features
  industry_nan[i] = All_features

## Seperate to 10 dataset based on industry


In [ ]:
# Create all industry Dataset at the same time
for i in df['industry'].unique():
  globals()['Data'+'_'+ i] = df.loc[df['industry'] == i] 
  globals()['Data'+'_'+ i] = globals()['Data'+'_'+ i][globals()['Data'+'_'+ i].columns & industry_nan[i]]
  globals()['Data'+'_'+ i] = globals()['Data'+'_'+ i].dropna()   # drop na

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  after removing the cwd from sys.path.


## Variables Selection Using K-Select

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, chi2

def Selection(X_t, y_t):
  best_features = SelectKBest(score_func = f_classif)      # chi square test assumes frequencies distribution and a frequency can't be a negative number
                                                            # Use f_classif isntead
  fitted_features = best_features.fit(X_t,y_t)
  feature_scores = pd.DataFrame(fitted_features.scores_)
  df_columns = pd.DataFrame(X_t.columns)
  predictor_Scores = pd.concat([df_columns,feature_scores],axis=1)
  predictor_Scores.columns = ['Predictor','Score']
  return predictor_Scores.nlargest(30,'Score')


In [ ]:
from sklearn.model_selection import train_test_split
from scipy import stats
import numpy as np

for i in df['industry'].unique():
  Industry_Data = globals()['Data'+'_'+ i]
  X_tr, X_te, y_tr, y_te = train_test_split(Industry_Data.iloc[:, 1:31], Industry_Data.trend, train_size = 0.7, random_state = 1)
  globals()['Result'+'_'+ i] = Selection(X_tr, y_tr)
  
  print(i)
  print(globals()['Result'+'_'+ i], '\n')

In [ ]:
for i in df['industry'].unique():
  globals()['Result'+'_'+ i].plot.bar(x = 'Predictor', title = i)

In [ ]:
for i in df['industry'].unique():
  selected_ratios = globals()['Result'+'_'+ i]['Predictor'][:15]  # Select Top X ratios
  selected_features = [i for i in selected_ratios] + Additional_features
  globals()[i+'_'+ 'Dataset'] = globals()['Data'+'_'+ i][globals()['Data'+'_'+ i].columns & selected_features]   # Select intersaction features


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  after removing the cwd from sys.path.


In [ ]:
for i in df['industry'].unique():
  print(i,":", len(globals()[i+'_'+ 'Dataset']))

Technology : 4941
Communication_Services : 1591
Consumer_Cyclical : 4442
Financial_Services : 5027
Healthcare : 4739
Consumer_Defensive : 2507
Energy : 1356
Industrials : 5080
Basic_Materials : 1526
Utilities : 2169
Real_Estate : 2233


In [ ]:
# Clean Outlier (90%)

for i in df['industry'].unique():
  Data = globals()[i+'_'+ 'Dataset']
  uncleaned_dataset = Data.iloc[:,1:-2]
  cols = uncleaned_dataset.columns
  Q1 = uncleaned_dataset[cols].quantile(0.1)
  Q3 = uncleaned_dataset[cols].quantile(0.9)
  IQR = Q3 - Q1

  globals()[i+'_'+ 'Dataset'] = Data [~((Data[cols] < (Q1 - 1.5 * IQR)) |(Data [cols] > (Q3 + 1.5 * IQR))).any(axis=1)]



# Build the model

In [ ]:
# 11 different Dataset
# Dataset: industry_dataset  ex: Technology_Dataset, Real_Estate_Dataset
# ratios ranking: Result_industry  ex: Result_Technology, Result_Real_Estate

In [ ]:
# Rows of each dataset
for i in df['industry'].unique():
  print(i,":", len(globals()[i+'_'+ 'Dataset']))

Technology : 3954
Communication_Services : 1170
Consumer_Cyclical : 3550
Financial_Services : 3959
Healthcare : 3762
Consumer_Defensive : 2109
Energy : 1055
Industrials : 3924
Basic_Materials : 1172
Utilities : 1708
Real_Estate : 1795


In [ ]:
# Simple ANN

def ANN(Epochs, Batch_size, X_train, y_train, X_test, y_test ):
  model = models.Sequential()
  model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer='l2'))
  model.add(Dropout(0.2))
  model.add(layers.Dense(128, activation='relu',kernel_regularizer='l2'))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))  


  model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=['accuracy'])
  #monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, verbose=1, mode='auto',
        # restore_best_weights=True)

  checkpoint = ModelCheckpoint('/content/drive/My Drive/WashU/courses/Seminar in Fintech/files/Code/Saved_Model/best_model.hdf5', monitor='val_accuracy', verbose=1,
      save_best_only=True, mode='auto', period=1)

  history = model.fit(X_train, y_train, epochs=Epochs, batch_size = Batch_size,    #delete step by epoch
                      validation_data=(X_test, y_test), callbacks=[checkpoint],    # delete monitor
                      verbose = 1)
  return history

In [ ]:
# Using Technology Industry
import keras
from keras import models
from keras import layers
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError
from keras.callbacks import ModelCheckpoint
from scipy import stats

trend = Technology_Dataset.iloc[:,-2:-1]
X_train, X_test, y_train, y_test = train_test_split(stats.zscore(Technology_Dataset.iloc[:, 1:11]), trend, train_size = 0.7)
#X_train, X_test, y_train, y_test = train_test_split(Technology_Dataset.iloc[:, 1:11], trend, train_size = 0.7)
history = ANN(400, 50, X_train, y_train, X_test, y_test )

In [ ]:
max(history.history['val_accuracy'])

0.676071047782898

In [ ]:
def ANN2(Epochs, Batch_size, X_train, y_train, X_test, y_test ):
  model = models.Sequential()
  model.add(layers.Dense(8, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))  


  model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  #monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, verbose=1, mode='auto',
        # restore_best_weights=True)

  checkpoint = ModelCheckpoint('/content/drive/My Drive/WashU/courses/Seminar in Fintech/files/Code/Saved_Model/best_model.hdf5', monitor='val_accuracy', verbose=1,
      save_best_only=True, mode='auto', period=1)

  history = model.fit(X_train, y_train, epochs=Epochs, batch_size = Batch_size,    #delete step by epoch
                      validation_data=(X_test, y_test), callbacks=[checkpoint],    # delete monitor
                      verbose = 1)
  return history

In [ ]:
trend = Technology_Dataset.iloc[:,-2:-1]
X_train, X_test, y_train, y_test = train_test_split(stats.zscore(Technology_Dataset.iloc[:, 1:11]), trend, train_size = 0.7)
#X_train, X_test, y_train, y_test = train_test_split(Technology_Dataset.iloc[:, 1:11], trend, train_size = 0.7)
history = ANN2(300, 50, X_train, y_train, X_test, y_test )

Epoch 1/300
61/70 [=========================>....] - ETA: 0s - loss: 0.6942 - accuracy: 0.5370
Epoch 1: val_accuracy improved from -inf to 0.64059, saving model to /content/drive/My Drive/WashU/courses/Seminar in Fintech/files/Code/Saved_Model/best_model.hdf5
70/70 [==============================] - 1s 9ms/step - loss: 0.6907 - accuracy: 0.5463 - val_loss: 0.6720 - val_accuracy: 0.6406
Epoch 2/300
64/70 [==========================>...] - ETA: 0s - loss: 0.6560 - accuracy: 0.6413
Epoch 2: val_accuracy did not improve from 0.64059
70/70 [==============================] - 0s 3ms/step - loss: 0.6565 - accuracy: 0.6391 - val_loss: 0.6637 - val_accuracy: 0.6406
Epoch 3/300
63/70 [==========================>...] - ETA: 0s - loss: 0.6513 - accuracy: 0.6422
Epoch 3: val_accuracy did not improve from 0.64059
70/70 [==============================] - 0s 3ms/step - loss: 0.6526 - accuracy: 0.6391 - val_loss: 0.6588 - val_accuracy: 0.6406
Epoch 4/300
65/70 [==========================>...] - ETA: 0s 

In [ ]:
max(history.history['val_accuracy'])

0.6608226299285889